In [1]:
'''

python exp-3-synchronize-frames-of-recorded-video.py inputFile.mp4 tranFile.csv outputFile noFrames noCells tempDir/

'''

'\n\npython exp-3-synchronize-frames-of-recorded-video.py inputFile.mp4 tranFile.csv outputFile noFrames noCells tempDir/\n\n'

In [2]:
# Include basics

import subprocess
import numpy as np
import cv2
import sys
import os
import time
import math


In [3]:
#Extract frames
WIDTH = 1920
HEIGHT = 1080
OUT_WIDTH = WIDTH
OUT_HEIGHT = HEIGHT
OUTIMG = True



inputFileName = "./OpenCamera/fractal.mp4"#Video recorded by the phone
tranFileName = "./OpenCamera/fractal_16090_5_split.csv"#CSV timestamp,groundTruth generated by the PC
outputFileName = "./OpenCamera/fractal"#'arr0'--->frame recorded by camera, arr1------>label


if len(sys.argv)>=4:
    inputFileName=sys.argv[1]
    tranFileName=sys.argv[2]
    outputFileName=sys.argv[3]
    
print((OUT_HEIGHT, OUT_WIDTH))

SYN = 0
FINE = 0



(1080, 1920)


In [4]:
frameTS = []
output = subprocess.Popen("ffprobe -v error  -show_entries frame=pkt_pts_time -select_streams v -of csv=p=0 " + inputFileName, shell=True, stdout=subprocess.PIPE).stdout.read()
#print('FFPROBE output',output)
stamps = output.decode("utf-8").rstrip().split('\n')

for ts in stamps:
    frameTS.append(float(ts))
print("Timestamps loaded")

Timestamps loaded


In [7]:
NO_FRAMES=20000
NO_CELLS=25
TEMP_DIR="tem/"

if len(sys.argv)>=7:
    NO_FRAMES=int(sys.argv[4])
    NO_CELLS=int(sys.argv[5])
    TEMP_DIR=sys.argv[6]


times = []
#pat55 = []
#pat33 = []
pat = []
print("Read tran file")
with open(tranFileName, 'r') as tranfile:
    line = tranfile.readline().rstrip()
    attrs = line.split(",")
    SYN = int(attrs[0])
    FIffprobeNE = float(attrs[1])
    for line in tranfile:
        attrs = line.rstrip().split(",")
        times.append(float(attrs[0]))
        pat.append(int(attrs[1]))

print("Num samples: " + str(len(times)))
violate = 0
imgset = []
tranindex = 1
index = 0
x_data = np.zeros((NO_FRAMES, int(OUT_HEIGHT/3), int(OUT_WIDTH/4), 3),dtype=np.uint8)
y_label = np.zeros((NO_FRAMES,NO_CELLS), dtype=np.uint8)
video = cv2.VideoCapture(inputFileName)
#video.set(cv2.CAP_PROP_POS_FRAMES, 8990)K Sahiti, LM Kalanadhabhatta

while(video.isOpened()):
    ret, frame = video.read()
    if ret:
        ts = frameTS[index] - frameTS[SYN] + FINE
        dist0 = times[tranindex-1] - times[0]
        dist1 = times[tranindex] - times[0]
        patindex = tranindex - 10
        if ts >= dist0 and ts <= dist1:
            imgset.append(frame)
        elif ts > dist1:
            if len(imgset) <= 2:
                violate = violate + 1
            if OUTIMG == True:
                if (tranindex < 100) or (tranindex > NO_FRAMES-200):
                    cv2.imwrite(TEMP_DIR + str(patindex) + ".jpg", imgset[len(imgset)-1])
            if (patindex >= 0) and (patindex < NO_FRAMES):
                x_data[patindex,:,:] = cv2.resize(imgset[len(imgset)-1][HEIGHT-OUT_HEIGHT:HEIGHT, 0:OUT_WIDTH,:], (int(OUT_WIDTH/4),int(OUT_HEIGHT/3)))
                tem = pat[tranindex]
                for i in range(NO_CELLS):
                    y_label[patindex,i] = tem & 0x01
                    tem = tem >> 1
            tranindex = tranindex + 1
            imgset = [frame]
            print("Total: " + str(patindex) + ", Violate: " + str(violate))
            if tranindex >= len(times):
                break
        index = index + 1
        
        
        '''if index==NO_FRAMES:
            break'''
        
    else:
        break
print("Releasing video")
video.release()
cv2.destroyAllWindows()


print(patindex)

'''noDataPoints=x_data.shape[0]
noBatches=math.ceil(noDataPoints/100)


for b in range(noBatches):
    bb=b*100
    np.savez(outputFileName+'{:02}'.format(x)+'.npz', X=x_data[bb:min(bb+100,x_data.shape[0])], Y=y_label[bb:min(bb+100,x_data.shape[0])])'''

np.savez(outputFileName+'.npz', X=x_data, Y=y_label)

print("Finished")



Read tran file
Num samples: 20016
Total: -9, Violate: 0
Total: -8, Violate: 0
Total: -7, Violate: 0
Total: -6, Violate: 0
Total: -5, Violate: 0
Total: -4, Violate: 0
Total: -3, Violate: 0
Total: -2, Violate: 0
Total: -1, Violate: 0
Total: 0, Violate: 0
Total: 1, Violate: 0
Total: 2, Violate: 0
Total: 3, Violate: 0
Total: 4, Violate: 0
Total: 5, Violate: 0
Total: 6, Violate: 0
Total: 7, Violate: 0
Total: 8, Violate: 0
Total: 9, Violate: 0
Total: 10, Violate: 0
Total: 11, Violate: 0
Total: 12, Violate: 0
Total: 13, Violate: 0
Total: 14, Violate: 0
Total: 15, Violate: 0
Total: 16, Violate: 0
Total: 17, Violate: 0
Total: 18, Violate: 0
Total: 19, Violate: 0
Total: 20, Violate: 0
Total: 21, Violate: 0
Total: 22, Violate: 0
Total: 23, Violate: 0
Total: 24, Violate: 0
Total: 25, Violate: 0
Total: 26, Violate: 0
Total: 27, Violate: 0
Total: 28, Violate: 0
Total: 29, Violate: 0
Total: 30, Violate: 0
Total: 31, Violate: 0
Total: 32, Violate: 0
Total: 33, Violate: 0
Total: 34, Violate: 0
Total: 3

Total: 356, Violate: 0
Total: 357, Violate: 0
Total: 358, Violate: 0
Total: 359, Violate: 0
Total: 360, Violate: 0
Total: 361, Violate: 0
Total: 362, Violate: 0
Total: 363, Violate: 0
Total: 364, Violate: 0
Total: 365, Violate: 0
Total: 366, Violate: 0
Total: 367, Violate: 0
Total: 368, Violate: 0
Total: 369, Violate: 0
Total: 370, Violate: 0
Total: 371, Violate: 0
Total: 372, Violate: 0
Total: 373, Violate: 0
Total: 374, Violate: 0
Total: 375, Violate: 0
Total: 376, Violate: 0
Total: 377, Violate: 0
Total: 378, Violate: 0
Total: 379, Violate: 0
Total: 380, Violate: 0
Total: 381, Violate: 0
Total: 382, Violate: 0
Total: 383, Violate: 0
Total: 384, Violate: 0
Total: 385, Violate: 0
Total: 386, Violate: 0
Total: 387, Violate: 0
Total: 388, Violate: 0
Total: 389, Violate: 0
Total: 390, Violate: 0
Total: 391, Violate: 0
Total: 392, Violate: 0
Total: 393, Violate: 0
Total: 394, Violate: 0
Total: 395, Violate: 0
Total: 396, Violate: 0
Total: 397, Violate: 0
Total: 398, Violate: 0
Total: 399,

Total: 714, Violate: 0
Total: 715, Violate: 0
Total: 716, Violate: 0
Total: 717, Violate: 0
Total: 718, Violate: 0
Total: 719, Violate: 0
Total: 720, Violate: 0
Total: 721, Violate: 0
Total: 722, Violate: 0
Total: 723, Violate: 0
Total: 724, Violate: 0
Total: 725, Violate: 0
Total: 726, Violate: 0
Total: 727, Violate: 0
Total: 728, Violate: 0
Total: 729, Violate: 0
Total: 730, Violate: 0
Total: 731, Violate: 0
Total: 732, Violate: 0
Total: 733, Violate: 0
Total: 734, Violate: 0
Total: 735, Violate: 0
Total: 736, Violate: 0
Total: 737, Violate: 0
Total: 738, Violate: 0
Total: 739, Violate: 0
Total: 740, Violate: 0
Total: 741, Violate: 0
Total: 742, Violate: 0
Total: 743, Violate: 0
Total: 744, Violate: 0
Total: 745, Violate: 0
Total: 746, Violate: 0
Total: 747, Violate: 0
Total: 748, Violate: 0
Total: 749, Violate: 0
Total: 750, Violate: 0
Total: 751, Violate: 0
Total: 752, Violate: 0
Total: 753, Violate: 0
Total: 754, Violate: 0
Total: 755, Violate: 0
Total: 756, Violate: 0
Total: 757,

Total: 1075, Violate: 0
Total: 1076, Violate: 0
Total: 1077, Violate: 0
Total: 1078, Violate: 0
Total: 1079, Violate: 0
Total: 1080, Violate: 0
Total: 1081, Violate: 0
Total: 1082, Violate: 0
Total: 1083, Violate: 0
Total: 1084, Violate: 0
Total: 1085, Violate: 0
Total: 1086, Violate: 0
Total: 1087, Violate: 0
Total: 1088, Violate: 0
Total: 1089, Violate: 0
Total: 1090, Violate: 0
Total: 1091, Violate: 0
Total: 1092, Violate: 0
Total: 1093, Violate: 0
Total: 1094, Violate: 0
Total: 1095, Violate: 0
Total: 1096, Violate: 0
Total: 1097, Violate: 0
Total: 1098, Violate: 0
Total: 1099, Violate: 0
Total: 1100, Violate: 0
Total: 1101, Violate: 0
Total: 1102, Violate: 0
Total: 1103, Violate: 0
Total: 1104, Violate: 0
Total: 1105, Violate: 0
Total: 1106, Violate: 0
Total: 1107, Violate: 0
Total: 1108, Violate: 0
Total: 1109, Violate: 0
Total: 1110, Violate: 0
Total: 1111, Violate: 0
Total: 1112, Violate: 0
Total: 1113, Violate: 0
Total: 1114, Violate: 0
Total: 1115, Violate: 0
Total: 1116, Vio

Total: 1429, Violate: 0
Total: 1430, Violate: 0
Total: 1431, Violate: 0
Total: 1432, Violate: 0
Total: 1433, Violate: 0
Total: 1434, Violate: 0
Total: 1435, Violate: 0
Total: 1436, Violate: 0
Total: 1437, Violate: 0
Total: 1438, Violate: 0
Total: 1439, Violate: 0
Total: 1440, Violate: 0
Total: 1441, Violate: 0
Total: 1442, Violate: 0
Total: 1443, Violate: 0
Total: 1444, Violate: 0
Total: 1445, Violate: 0
Total: 1446, Violate: 0
Total: 1447, Violate: 0
Total: 1448, Violate: 0
Total: 1449, Violate: 0
Total: 1450, Violate: 0
Total: 1451, Violate: 0
Total: 1452, Violate: 0
Total: 1453, Violate: 0
Total: 1454, Violate: 0
Total: 1455, Violate: 0
Total: 1456, Violate: 0
Total: 1457, Violate: 0
Total: 1458, Violate: 0
Total: 1459, Violate: 0
Total: 1460, Violate: 0
Total: 1461, Violate: 0
Total: 1462, Violate: 0
Total: 1463, Violate: 0
Total: 1464, Violate: 0
Total: 1465, Violate: 0
Total: 1466, Violate: 0
Total: 1467, Violate: 0
Total: 1468, Violate: 0
Total: 1469, Violate: 0
Total: 1470, Vio

...

Total: 18511, Violate: 1
Total: 18512, Violate: 1
Total: 18513, Violate: 1
Total: 18514, Violate: 1
Total: 18515, Violate: 1
Total: 18516, Violate: 1
Total: 18517, Violate: 1
Total: 18518, Violate: 1
Total: 18519, Violate: 1
Total: 18520, Violate: 1
Total: 18521, Violate: 1
Total: 18522, Violate: 1
Total: 18523, Violate: 1
Total: 18524, Violate: 1
Total: 18525, Violate: 1
Total: 18526, Violate: 1
Total: 18527, Violate: 1
Total: 18528, Violate: 1
Total: 18529, Violate: 1
Total: 18530, Violate: 1
Total: 18531, Violate: 1
Total: 18532, Violate: 1
Total: 18533, Violate: 1
Total: 18534, Violate: 1
Total: 18535, Violate: 1
Total: 18536, Violate: 1
Total: 18537, Violate: 1
Total: 18538, Violate: 1
Total: 18539, Violate: 1
Total: 18540, Violate: 1
Total: 18541, Violate: 1
Total: 18542, Violate: 1
Total: 18543, Violate: 1
Total: 18544, Violate: 1
Total: 18545, Violate: 1
Total: 18546, Violate: 1
Total: 18547, Violate: 1
Total: 18548, Violate: 1
Total: 18549, Violate: 1
Total: 18550, Violate: 1


Total: 18840, Violate: 1
Total: 18841, Violate: 1
Total: 18842, Violate: 1
Total: 18843, Violate: 1
Total: 18844, Violate: 1
Total: 18845, Violate: 1
Total: 18846, Violate: 1
Total: 18847, Violate: 1
Total: 18848, Violate: 1
Total: 18849, Violate: 1
Total: 18850, Violate: 1
Total: 18851, Violate: 1
Total: 18852, Violate: 1
Total: 18853, Violate: 1
Total: 18854, Violate: 1
Total: 18855, Violate: 1
Total: 18856, Violate: 1
Total: 18857, Violate: 1
Total: 18858, Violate: 1
Total: 18859, Violate: 1
Total: 18860, Violate: 1
Total: 18861, Violate: 1
Total: 18862, Violate: 1
Total: 18863, Violate: 1
Total: 18864, Violate: 1
Total: 18865, Violate: 1
Total: 18866, Violate: 1
Total: 18867, Violate: 1
Total: 18868, Violate: 1
Total: 18869, Violate: 1
Total: 18870, Violate: 1
Total: 18871, Violate: 1
Total: 18872, Violate: 1
Total: 18873, Violate: 1
Total: 18874, Violate: 1
Total: 18875, Violate: 1
Total: 18876, Violate: 1
Total: 18877, Violate: 1
Total: 18878, Violate: 1
Total: 18879, Violate: 1


Total: 19171, Violate: 2
Total: 19172, Violate: 2
Total: 19173, Violate: 2
Total: 19174, Violate: 2
Total: 19175, Violate: 2
Total: 19176, Violate: 2
Total: 19177, Violate: 2
Total: 19178, Violate: 2
Total: 19179, Violate: 2
Total: 19180, Violate: 2
Total: 19181, Violate: 2
Total: 19182, Violate: 2
Total: 19183, Violate: 2
Total: 19184, Violate: 2
Total: 19185, Violate: 2
Total: 19186, Violate: 2
Total: 19187, Violate: 2
Total: 19188, Violate: 2
Total: 19189, Violate: 2
Total: 19190, Violate: 2
Total: 19191, Violate: 2
Total: 19192, Violate: 2
Total: 19193, Violate: 2
Total: 19194, Violate: 2
Total: 19195, Violate: 2
Total: 19196, Violate: 2
Total: 19197, Violate: 2
Total: 19198, Violate: 2
Total: 19199, Violate: 2
Total: 19200, Violate: 2
Total: 19201, Violate: 2
Total: 19202, Violate: 2
Total: 19203, Violate: 2
Total: 19204, Violate: 2
Total: 19205, Violate: 2
Total: 19206, Violate: 2
Total: 19207, Violate: 2
Total: 19208, Violate: 2
Total: 19209, Violate: 2
Total: 19210, Violate: 2


Total: 19512, Violate: 2
Total: 19513, Violate: 2
Total: 19514, Violate: 2
Total: 19515, Violate: 2
Total: 19516, Violate: 2
Total: 19517, Violate: 2
Total: 19518, Violate: 2
Total: 19519, Violate: 2
Total: 19520, Violate: 2
Total: 19521, Violate: 2
Total: 19522, Violate: 2
Total: 19523, Violate: 2
Total: 19524, Violate: 2
Total: 19525, Violate: 2
Total: 19526, Violate: 2
Total: 19527, Violate: 2
Total: 19528, Violate: 2
Total: 19529, Violate: 2
Total: 19530, Violate: 2
Total: 19531, Violate: 2
Total: 19532, Violate: 2
Total: 19533, Violate: 2
Total: 19534, Violate: 2
Total: 19535, Violate: 2
Total: 19536, Violate: 2
Total: 19537, Violate: 2
Total: 19538, Violate: 2
Total: 19539, Violate: 2
Total: 19540, Violate: 2
Total: 19541, Violate: 2
Total: 19542, Violate: 2
Total: 19543, Violate: 2
Total: 19544, Violate: 2
Total: 19545, Violate: 2
Total: 19546, Violate: 2
Total: 19547, Violate: 2
Total: 19548, Violate: 2
Total: 19549, Violate: 2
Total: 19550, Violate: 2
Total: 19551, Violate: 2


Total: 19845, Violate: 2
Total: 19846, Violate: 2
Total: 19847, Violate: 2
Total: 19848, Violate: 2
Total: 19849, Violate: 2
Total: 19850, Violate: 2
Total: 19851, Violate: 2
Total: 19852, Violate: 2
Total: 19853, Violate: 2
Total: 19854, Violate: 2
Total: 19855, Violate: 2
Total: 19856, Violate: 2
Total: 19857, Violate: 2
Total: 19858, Violate: 2
Total: 19859, Violate: 2
Total: 19860, Violate: 2
Total: 19861, Violate: 2
Total: 19862, Violate: 2
Total: 19863, Violate: 2
Total: 19864, Violate: 2
Total: 19865, Violate: 2
Total: 19866, Violate: 2
Total: 19867, Violate: 2
Total: 19868, Violate: 2
Total: 19869, Violate: 2
Total: 19870, Violate: 2
Total: 19871, Violate: 2
Total: 19872, Violate: 2
Total: 19873, Violate: 2
Total: 19874, Violate: 2
Total: 19875, Violate: 2
Total: 19876, Violate: 2
Total: 19877, Violate: 2
Total: 19878, Violate: 2
Total: 19879, Violate: 2
Total: 19880, Violate: 2
Total: 19881, Violate: 2
Total: 19882, Violate: 2
Total: 19883, Violate: 2
Total: 19884, Violate: 2


In [ ]:
#np.savez(outputFileName+'.npz', X=x_data, Y=y_label)
